<a href="https://colab.research.google.com/github/Bovi-analytics/Hut-et-al-2022/blob/main/DefinitiveKNMIcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for  KNMI data

This is a Google Colab Notebook. It has a google machine in the back which will activate the moment you click the play button from the first cell. You can run cells separately, or all after each other. It allows you to prototype much easier then using the PyCharm.

The notebook can be shared and editted by multiple people. If you want to work in your own version, you can make a copy of the notebook first.

#GDrive
You need to make a link with the data. This is all stored in your google drive , or in the case of this notebook a bovi-analytics folder.

First run the cell underneath, then open the link in the output and copy the code underneath. That way, the Google Drive is connected.

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount = True) 

Mounted at /content/gdrive/


In [ ]:
%reload_ext rpy2.ipython

I added this piece of code to copy the Google Drive documents to the local Google machine. This was because of gdrive timeouts
https://research.google.com/colaboratory/faq.html#drive-timeout

# Load libraries

In [ ]:
%%R
lib_loc <- "/content/drive/Shareddrives/Bovi-Analytics/R-lib/"

In [ ]:
%%R
install.packages("trees", lib=lib_loc)

R[write to console]: Warning in install.packages("trees", lib = lib_loc) :
R[write to console]: 
 
R[write to console]:  'lib = "/content/drive/Shareddrives/Bovi-Analytics/R-lib/"' is not writable



Would you like to use a personal library instead? (yes/No/cancel) yes
Would you like to create a personal library
‘~/R/x86_64-pc-linux-gnu-library/4.1’
to install packages into? (yes/No/cancel) yes


In [ ]:
%%R
lib_loc="~/R/x86_64-pc-linux-gnu-library/4.1"

In [ ]:
%%R
install.packages("dplyr", lib=lib_loc)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.0.7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 852103 bytes (832 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]:

In [ ]:
%%R
package_list <- c("dplyr")

for (pkg in package_list) {
  if (pkg %in% rownames(.packages()) == FALSE)
  {library(pkg, lib.loc = lib_loc, character.only = TRUE)}

}

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
# Load general libraries
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy.stats import mode

# Load directories to work with

In [ ]:
# directories waarmee je gaat werken definiëren
base_dir = '/content/gdrive/Shareddrives/Bovi-Analytics/Projects/JosjeScheurwater/ProjectHeatStressSensor/data'
input_dir = '/in/THIData/'
output_dir = '/out/'

input_path = base_dir + input_dir
output_path = base_dir + output_dir


In [ ]:
!ls /content/gdrive/'Shared drives'/Bovi-Analytics/Projects/JosjeScheurwater/ProjectHeatStressSensor/data/in/THIData/

KNMI_DATA.csv  Knmi_stations.csv


In [ ]:
df_stations = pd.read_csv(input_path + 'Knmi_stations.csv', header=0, sep = ";", decimal='.', usecols=['HerdIdentifier','Station_number'])
df_stations= df_stations.rename(columns= {'Station_number':'Location'})

# Read THI datafile

## Features

In [ ]:
df_thi = pd.read_csv(input_path + 'KNMI_DATA.csv', header=0, sep = ";", decimal='.')
df_thi['temperature']=df_thi.temperature.astype(float)
df_thi['THI']=df_thi.THI.astype(float)
df_thi['humidity']=df_thi.humidity.astype(float)
df_thi['Date'] = pd.to_datetime(df_thi['date'], format='%Y%m%d')
df_thi["Day"] = df_thi['Date'].map(lambda x: x.day)
df_thi["Month"] = df_thi['Date'].map(lambda x: x.month)
df_thi["Year"] = df_thi['Date'].map(lambda x: x.year)
df_thi= df_thi.rename(columns= {'location':'Location', 'temperature': 'Temperature', 'humidity': 'Humidity', 'hour': 'Hour'})
del df_thi['Hour']
del df_thi['date']
df_thi.head(5)

,Location,Temperature,Humidity,THI,Date,Day,Month,Year
0,275,0.4,99.0,32.86,2017-01-01,1,1,2017
1,275,0.3,99.0,32.68,2017-01-01,1,1,2017
2,275,0.2,99.0,32.50,2017-01-01,1,1,2017
3,275,0.0,99.0,32.14,2017-01-01,1,1,2017
4,275,0.0,99.0,32.14,2017-01-01,1,1,2017


## Create df_KNMI_ym

In [ ]:
df_KNMI_ym=df_thi.groupby(['Year','Month'], as_index=False).agg(
    THI_mean= ('THI','mean'),
    THI_min= ('THI','min'),
    THI_max= ('THI','max'),
    Temp_mean= ('Temperature','mean'),
    Temp_min= ('Temperature','min'),
    Temp_max= ('Temperature','max'),
    Hum_mean= ('Humidity','mean'),
    Hum_min= ('Humidity','min'),
    Hum_max= ('Humidity','max')
    )

In [ ]:
df_KNMI_ym.to_csv(output_path + 'df_KNMI_ym', sep = ";", decimal='.', index=False)

## Create df_KNMI_date

In [ ]:
df_KNMI_date=df_thi.groupby(['Year','Month', 'Day','Date'], as_index=False).agg(
    THI_mean= ('THI','mean'),
    THI_min= ('THI','min'),
    THI_max= ('THI','max'),
    Temp_mean= ('Temperature','mean'),
    Temp_min= ('Temperature','min'),
    Temp_max= ('Temperature','max'),
    Hum_mean= ('Humidity','mean'),
    Hum_min= ('Humidity','min'),
    Hum_max= ('Humidity','max')
    )

In [ ]:
df_KNMI_date.to_csv(output_path + 'df_KNMI_date', sep = ";", decimal='.', index=False)

## Create df_KNMI_herd

In [ ]:
df_THI_merged = pd.merge(df_stations, df_thi, on=['Location'], how='outer')

In [ ]:
def thi_counter_from (col,t):
  return col[col>=t].count()

In [ ]:
def thi_counter_till (col,t):
  return col[col<t].count()

In [ ]:
df_KNMI_herd=df_THI_merged.groupby(['HerdIdentifier','Date','Year','Month','Day'], as_index=False).agg(
    THI_mean= ('THI','mean'),
    THI_min= ('THI','min'),
    THI_max= ('THI','max'),
    Temp_mean= ('Temperature','mean'),
    Temp_min= ('Temperature','min'),
    Temp_max= ('Temperature','max'),
    Hum_mean= ('Humidity','mean'),
    Hum_min= ('Humidity','min'),
    Hum_max= ('Humidity','max'),
    THI_count60= ('THI', lambda x: thi_counter_from(x,t=60)),
    THI_count72= ('THI', lambda x: thi_counter_from(x,t=72)),
    THI_counttill60= ('THI', lambda x: thi_counter_till(x,t=60)),
    Temp_count16= ('Temperature', lambda x: thi_counter_from(x,t=16)),
    Temp_count20= ('Temperature', lambda x: thi_counter_from(x,t=20)),
    Temp_counttill16= ('Temperature', lambda x: thi_counter_till(x,t=16)),
    )

In [ ]:
df_KNMI_herd['THI_mean_yesterday']= df_KNMI_herd['THI_mean'].shift(1)
df_KNMI_herd['THI_mean_2daysbefore']= df_KNMI_herd['THI_mean'].shift(2)
df_KNMI_herd['Temp_mean_yesterday']= df_KNMI_herd['Temp_mean'].shift(1)
df_KNMI_herd['Temp_mean_2daysbefore']= df_KNMI_herd['Temp_mean'].shift(2)

### Create_df_KNMI_herd

In [ ]:
df_KNMI_herd.to_csv(output_path + 'df_KNMI_herd.csv', sep = ";", decimal='.', index=False)

### Create df_KNMI_per3

In [ ]:
%%R
df_KNMI_herd <-read.csv(file= '/content/gdrive/Shareddrives/Bovi-Analytics/Projects/JosjeScheurwater/ProjectHeatStressSensor/data/out/df_KNMI_herd.csv', sep = ";" )

In [ ]:
%%R
df_KNMI_3 <- df_KNMI_herd %>%
  dplyr::mutate (ThiMean1daybeforeCat= case_when(THI_mean_yesterday < 30 ~ 0,
                                   THI_mean_yesterday < 57 ~ 1, 
                                   THI_mean_yesterday < 60 ~ 2,
                                   THI_mean_yesterday < 63 ~ 3,
                                   THI_mean_yesterday < 66 ~ 4,      
                                   THI_mean_yesterday < 69 ~ 5,  
                                   THI_mean_yesterday < 72 ~ 6,                                                                                                 
                                   THI_mean_yesterday >= 72 ~ 7)) %>%
  dplyr::mutate (ThiMean2daysbeforeCat= case_when(THI_mean_2daysbefore < 30 ~ 0,
                                   THI_mean_2daysbefore < 57 ~ 1, 
                                   THI_mean_2daysbefore < 60 ~ 2,
                                   THI_mean_2daysbefore < 63 ~ 3,
                                   THI_mean_2daysbefore < 66 ~ 4,      
                                   THI_mean_2daysbefore < 69 ~ 5,  
                                   THI_mean_2daysbefore < 72 ~ 6,                                                                                                 
                                   THI_mean_2daysbefore >= 72 ~ 7)) %>%
  dplyr::mutate (ThiMeanCat= case_when(THI_mean < 30 ~ 0,
                                   THI_mean < 57 ~ 1, 
                                   THI_mean < 60 ~ 2,
                                   THI_mean < 63 ~ 3,
                                   THI_mean < 66 ~ 4,      
                                   THI_mean < 69 ~ 5,  
                                   THI_mean < 72 ~ 6,                                                                                                 
                                   THI_mean >= 72 ~ 7)) %>%                               
  dplyr::mutate (ThiMinCat= case_when(THI_min < 35 ~ 0,
                                   THI_min < 45 ~ 1, 
                                   THI_min < 48 ~ 2,
                                   THI_min < 51 ~ 3,
                                   THI_min < 54 ~ 4,
                                   THI_min < 57 ~ 5,
                                   THI_min < 60 ~ 6,
                                   THI_min >= 60 ~ 7)) %>%    
  dplyr::mutate (ThiMaxCat= case_when(THI_max < 30 ~ 0,
                                   THI_max < 57 ~ 1, 
                                   THI_max < 60 ~ 2,
                                   THI_max < 63 ~ 3,
                                   THI_max < 66 ~ 4,   
                                   THI_max < 69 ~ 5,                             
                                   THI_max < 72 ~ 6, 
                                   THI_max >= 72 ~ 7)) %>%                                                                   
  dplyr::mutate (TempMeanCat = case_when(Temp_mean < 0 ~ 0, 
                                   Temp_mean < 13 ~ 1,
                                   Temp_mean < 16 ~ 2,
                                   Temp_mean < 19 ~ 3, 
                                   Temp_mean < 22 ~ 4,                               
                                   Temp_mean < 25 ~ 5,
                                   Temp_mean < 28 ~ 6,                              
                                   Temp_mean >= 28 ~ 7)) %>%
  dplyr::mutate (TempMinCat = case_when(Temp_min < 0 ~ 0, 
                                   Temp_min < 4 ~ 1,
                                   Temp_min < 7 ~ 2,
                                   Temp_min < 10 ~ 3, 
                                   Temp_min < 13 ~ 4, 
                                   Temp_min < 16 ~ 5,
                                   Temp_min < 20 ~ 6,                                                                    
                                   Temp_min >= 20 ~ 7)) %>%
  dplyr::mutate (TempMaxCat = case_when(Temp_max < 0 ~ 0, 
                                   Temp_max < 13 ~ 1,
                                   Temp_max < 16 ~ 2,
                                   Temp_max < 19 ~ 3, 
                                   Temp_max < 22 ~ 4,                               
                                   Temp_max < 25 ~ 5,
                                   Temp_max < 28 ~ 6,                              
                                   Temp_max >= 28 ~ 7)) %>%   
    dplyr::mutate (TempMean1daybeforeCat= case_when(Temp_mean_yesterday < 0 ~ 0,
                                   Temp_mean_yesterday < 12 ~ 1, 
                                   Temp_mean_yesterday < 16 ~ 2,
                                   Temp_mean_yesterday < 19 ~ 3,
                                   Temp_mean_yesterday < 22 ~ 4,      
                                   Temp_mean_yesterday < 25 ~ 5,  
                                   Temp_mean_yesterday < 28 ~ 6,                                                                                                 
                                   Temp_mean_yesterday >= 28 ~ 7)) %>%
  dplyr::mutate (TempMean2daysbeforeCat= case_when(Temp_mean_2daysbefore < 0 ~ 0,
                                   Temp_mean_2daysbefore < 12 ~ 1, 
                                   Temp_mean_2daysbefore < 16 ~ 2,
                                   Temp_mean_2daysbefore < 19 ~ 3,
                                   Temp_mean_2daysbefore < 22 ~ 4,      
                                   Temp_mean_2daysbefore < 25 ~ 5,  
                                   Temp_mean_2daysbefore < 28 ~ 6,                                                                                                 
                                   Temp_mean_2daysbefore >= 28 ~ 7)) %>%
  dplyr::mutate (Thi6Hrs60 = case_when(THI_counttill60 < 6 ~ 0, 
                                   THI_counttill60 >= 6 ~ 1)) %>%     
  dplyr::mutate (Temp6Hrs16 = case_when(Temp_counttill16 < 6 ~ 0, 
                                   Temp_counttill16 >= 6 ~ 1))

In [ ]:
%%R
write.csv(df_KNMI_3, '/content/gdrive/Shareddrives/Bovi-Analytics/Projects/JosjeScheurwater/ProjectHeatStressSensor/data/out/df_KNMI_per3.csv', sep = ";")

### Create df_KNMI_per4

In [ ]:
%%R
df_KNMI_per4 <- df_KNMI_herd %>%
 dplyr::mutate (ThiMean1daybeforeCat= case_when(THI_mean_yesterday < 30 ~ 0,
                                   THI_mean_yesterday < 56 ~ 1, 
                                   THI_mean_yesterday < 60 ~ 2,
                                   THI_mean_yesterday < 64 ~ 3,
                                   THI_mean_yesterday < 68 ~ 4,
                                   THI_mean_yesterday < 72 ~ 5,                                
                                   THI_mean_yesterday >= 72 ~ 6)) %>%
 dplyr::mutate (ThiMean2daysbeforeCat= case_when(THI_mean_2daysbefore < 30 ~ 0,
                                   THI_mean_2daysbefore < 56 ~ 1, 
                                   THI_mean_2daysbefore < 60 ~ 2,
                                   THI_mean_2daysbefore < 64 ~ 3,
                                   THI_mean_2daysbefore < 68 ~ 4,
                                   THI_mean_2daysbefore < 72 ~ 5,                                
                                   THI_mean_2daysbefore >= 72 ~ 6)) %>%                                   
 dplyr::mutate (ThiMeanCat= case_when(THI_mean < 30 ~ 0,
                                   THI_mean < 56 ~ 1, 
                                   THI_mean < 60 ~ 2,
                                   THI_mean < 64 ~ 3,
                                   THI_mean < 68 ~ 4,
                                   THI_mean < 72 ~ 5,                                
                                   THI_mean >= 72 ~ 6)) %>%                                                                        
 dplyr::mutate (ThiMinCat= case_when(THI_min < 30 ~ 0, 
                                   THI_min < 48 ~ 1,
                                   THI_min < 52 ~ 2,
                                   THI_min < 56 ~ 3,                                
                                   THI_min < 60 ~ 4,
                                   THI_min < 64 ~ 5,                                
                                   THI_min >= 64 ~ 6)) %>%    
  dplyr::mutate (ThiMaxCat= case_when(THI_max < 30 ~ 0,
                                   THI_max < 56 ~ 1, 
                                   THI_max < 60 ~ 2,
                                   THI_max < 64 ~ 3,
                                   THI_max < 68 ~ 4,   
                                   THI_max < 72 ~ 5,                             
                                   THI_max >= 72 ~ 6)) %>%                                                                   
  dplyr::mutate (TempMeanCat = case_when(Temp_mean < 0 ~ 0, 
                                   Temp_mean < 12 ~ 1,
                                   Temp_mean < 16 ~ 2,
                                   Temp_mean < 20 ~ 3, 
                                   Temp_mean < 24 ~ 4, 
                                   Temp_mean < 28 ~ 5,                               
                                   Temp_mean >= 28 ~ 6)) %>%
  dplyr::mutate (TempMean1daybeforeCat = case_when(Temp_mean_yesterday < 0 ~ 0, 
                                   Temp_mean_yesterday < 12 ~ 1,
                                   Temp_mean_yesterday < 16 ~ 2,
                                   Temp_mean_yesterday < 20 ~ 3, 
                                   Temp_mean_yesterday < 24 ~ 4, 
                                   Temp_mean_yesterday < 28 ~ 5,                               
                                   Temp_mean_yesterday >= 28 ~ 6)) %>%   
 dplyr::mutate (TempMean2daysbeforeCat= case_when(Temp_mean_2daysbefore < 0 ~ 0,
                                   Temp_mean_2daysbefore < 12 ~ 1, 
                                   Temp_mean_2daysbefore < 16 ~ 2,
                                   Temp_mean_2daysbefore < 20 ~ 3,
                                   Temp_mean_2daysbefore < 24 ~ 4,
                                   Temp_mean_2daysbefore < 28 ~ 5,                                
                                   Temp_mean_2daysbefore >= 28 ~ 6)) %>%                                                                     
  dplyr::mutate (TempMinCat = case_when(Temp_min < 0 ~ 0, 
                                   Temp_min < 4 ~ 1,
                                   Temp_min < 8 ~ 2, 
                                   Temp_min < 12 ~ 3, 
                                   Temp_min < 16 ~ 4,  
                                   Temp_min < 20 ~ 5,                              
                                   Temp_min >= 20 ~ 6)) %>%
  dplyr::mutate (TempMaxCat = case_when(Temp_max < 0 ~ 0, 
                                   Temp_max < 12 ~ 1,
                                   Temp_max < 16 ~ 2, 
                                   Temp_max < 20 ~ 3,
                                   Temp_max < 24 ~ 4,                                
                                   Temp_max < 28 ~ 5,
                                   Temp_max >= 28 ~ 6)) %>%    
  dplyr::mutate (Thi6Hrs60 = case_when(THI_counttill60 < 6 ~ 0, 
                                   THI_counttill60 >= 6 ~ 1)) %>%     
  dplyr::mutate (Temp6Hrs16 = case_when(Temp_counttill16 < 6 ~ 0, 
                                   Temp_counttill16 >= 6 ~ 1))                                                                    

In [ ]:
%%R
write.csv(df_KNMI_per4, '/content/gdrive/Shareddrives/Bovi-Analytics/Projects/JosjeScheurwater/ProjectHeatStressSensor/data/out/df_KNMI_per4.csv', sep = ";")

### Create df_KNMI_per5

In [ ]:
%%R
df_KNMI_per5 <- df_KNMI_herd%>%
  dplyr::mutate (ThiMeanCat= case_when(THI_mean < 30 ~ 0,
                                   THI_mean < 55 ~ 1, 
                                   THI_mean < 60 ~ 2,
                                   THI_mean < 65 ~ 3,
                                   THI_mean < 70 ~ 4,                                
                                   THI_mean >= 70 ~ 5)) %>%
  dplyr::mutate (ThiMean1daybeforeCat= case_when(THI_mean_yesterday < 30 ~ 0,
                                   THI_mean_yesterday < 55 ~ 1, 
                                   THI_mean_yesterday < 60 ~ 2,
                                   THI_mean_yesterday < 65 ~ 3,
                                   THI_mean_yesterday < 70 ~ 4,                                
                                   THI_mean_yesterday >= 70 ~ 5)) %>%
  dplyr::mutate (ThiMean2daysbeforeCat= case_when(THI_mean_2daysbefore < 30 ~ 0,
                                   THI_mean_2daysbefore < 55 ~ 1, 
                                   THI_mean_2daysbefore < 60 ~ 2,
                                   THI_mean_2daysbefore < 65 ~ 3,
                                   THI_mean_2daysbefore < 70 ~ 4,                                
                                   THI_mean_2daysbefore >= 70 ~ 5)) %>%                                                                      
 dplyr::mutate (ThiMinCat= case_when(THI_min < 30 ~ 0,
                                   THI_min < 45 ~ 1, 
                                   THI_min < 50 ~ 2,
                                   THI_min < 55 ~ 3,
                                   THI_min < 60 ~ 4,                                
                                   THI_min >= 60 ~ 5)) %>%    
  dplyr::mutate (ThiMaxCat= case_when(THI_max < 30 ~ 0,
                                   THI_max < 55 ~ 1, 
                                   THI_max < 60 ~ 2,
                                   THI_max < 65 ~ 3,
                                   THI_max < 70 ~ 4,   
                                   THI_max >= 70 ~ 5)) %>%                                                                   
  dplyr::mutate (TempMeanCat = case_when(Temp_mean < 0 ~ 0, 
                                   Temp_mean < 10 ~ 1,
                                   Temp_mean < 15 ~ 2,
                                   Temp_mean < 20 ~ 3, 
                                   Temp_mean < 25 ~ 4,                           
                                   Temp_mean >= 25 ~ 5)) %>%
  dplyr::mutate (TempMean1daybeforeCat = case_when(Temp_mean_yesterday < 0 ~ 0, 
                                   Temp_mean_yesterday < 10 ~ 1,
                                   Temp_mean_yesterday < 15 ~ 2,
                                   Temp_mean_yesterday < 20 ~ 3, 
                                   Temp_mean_yesterday < 25 ~ 4,                           
                                   Temp_mean_yesterday >= 25 ~ 5)) %>%
  dplyr::mutate (TempMean2daysbeforeCat = case_when(Temp_mean_2daysbefore < 0 ~ 0, 
                                   Temp_mean_2daysbefore < 10 ~ 1,
                                   Temp_mean_2daysbefore < 15 ~ 2,
                                   Temp_mean_2daysbefore < 20 ~ 3, 
                                   Temp_mean_2daysbefore < 25 ~ 4,                           
                                   Temp_mean_2daysbefore >= 25 ~ 5)) %>%                                                                      
  dplyr::mutate (TempMinCat = case_when(Temp_min < 0 ~ 0, 
                                   Temp_min < 5 ~ 1,
                                   Temp_min < 10 ~ 2, 
                                   Temp_min < 15 ~ 3, 
                                   Temp_min < 20 ~ 4,                                 
                                   Temp_min >= 20 ~ 5)) %>%
  dplyr::mutate (TempMaxCat = case_when(Temp_max < 0 ~ 0, 
                                   Temp_max < 10 ~ 1,
                                   Temp_max < 15 ~ 2, 
                                   Temp_max < 20 ~ 3,
                                   Temp_max < 25 ~ 4,                                
                                   Temp_max >= 25 ~ 5)) %>%    
  dplyr::mutate (Thi6Hrs60 = case_when(THI_counttill60 < 6 ~ 0, 
                                   THI_counttill60 >= 6 ~ 1)) %>%     
  dplyr::mutate (Temp6Hrs16 = case_when(Temp_counttill16 < 6 ~ 0, 
                                   Temp_counttill16 >= 6 ~ 1))  

In [ ]:
%%R
write.csv(df_KNMI_per5, '/content/gdrive/Shareddrives/Bovi-Analytics/Projects/JosjeScheurwater/ProjectHeatStressSensor/data/out/df_KNMI_per5.csv', sep = ";")